In [1]:
import pandas as pd

In [2]:
ruta="/home/donsson/proyectos/VENTAS/Referencias Promo Autoparts.xlsx"

df = pd.read_excel(ruta)

In [3]:
df.head()

,Referencia interna,Nombre,Categoría interna,Clase,Precio de venta,Cantidad a mano
0,DCE00454189,"G454 FILTRO COMBUSTIBLE SUBARU, TOYOTA.",Industrias Donsson / Filtros / Autoparts / Imp...,C,4500,103
1,DLE00909189,G909 FILTRO ACEITE - RENAULT KOLEOS Y MASTER,Industrias Donsson / Filtros / Autoparts / Imp...,C,18600,104
2,DAE06784189,"GA784 FILTRO AIRE DAIHATSU TERIOS, TOYOTA COROLLA",Industrias Donsson / Filtros / Autoparts / Imp...,C,22000,104
3,DCS00740189,GS740 FILTRO COMBUSTIBLE - HINO,Industrias Donsson / Filtros / Autoparts / Imp...,C,75000,105
4,DAE00706189,GA706 FILTRO AIRE - NISSAN FRONTIER D22 ANTERI...,Industrias Donsson / Filtros / Autoparts / Imp...,C,41000,109
